In [56]:
#!pip install seaborn
#!pip install openpyxl


In [57]:
import pandas as pd
import numpy as np
import random as rd
import time
import csv
import seaborn as sbs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split ,GridSearchCV
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [58]:
file_data = 'DS_Agua_2017_2022_por_ponto.csv'
path_name='data/'
path_name_results='results/'
file_result = 'Result_MLP_Model_Day.csv'

In [59]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')


In [60]:
dataset.head()      

,DT_MEDICAO_HORA,PRECIPITACAO,PRESSAO_ATMOSFERICA,TEMPERATURA_DO_AR_C,UMIDADE_RELATIVA_DO_AR,VELOCIDADE_VENTO,SK_PONTO,VL_MEDICAO
1,2017-01-01 00:00:00,0.0,1009.5,25.9,75.0,1.3,5,938.690000
2,2017-01-01 00:00:00,0.0,1009.5,25.9,75.0,1.3,6,297.107800
3,2017-01-01 00:00:00,0.0,1009.5,25.9,75.0,1.3,7,544.594400
4,2017-01-01 00:00:00,0.0,1009.5,25.9,75.0,1.3,11,816.790594
5,2017-01-01 00:00:00,0.0,1009.5,25.9,75.0,1.3,12,336.596097


In [61]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
SK_PONTO                  0
VL_MEDICAO                0
dtype: int64

In [62]:
def salvar_resultado(sk_ponto, ds_best_param, MSE, RMSE, MAE, MAPE, Duration):
  #Script to write training cycle results
  data = [sk_ponto, ds_best_param, MSE, RMSE, MAE, MAPE, Duration]
  fields = ['SK_PONTO','Best Params','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a") as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['SK_PONTO','Best Params','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w") as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [63]:
#crop date time to date 
dataset['DATA']=dataset['DT_MEDICAO_HORA'].str.split(' ').str[0]
dataset.drop(['DT_MEDICAO_HORA'],axis=1, inplace=True)

In [64]:
# Grouping data for Day
# #cols = 'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'
dataset = dataset.groupby(['SK_PONTO','DATA'], as_index=False).agg({'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'})
#df_result.rename(columns=cols)

In [65]:
def previsao_MLP(sk_ponto, dataset):
  # dataframe tratament
  df = pd.DataFrame()
  df['VL_MEDICAO']=dataset['VL_MEDICAO']  

  # time serire transform - shit 1 step time
  seqtemp=1
  df['vl-1']=dataset['VL_MEDICAO'].shift(seqtemp)
  df['tp-1']=dataset['TEMPERATURA_DO_AR_C']
  df['pr-1']=dataset['PRESSAO_ATMOSFERICA']
  df['vv-1']=dataset['VELOCIDADE_VENTO']
  df['ur-1']=dataset['UMIDADE_RELATIVA_DO_AR']
  df['ch-1']=dataset['PRECIPITACAO']  

  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(df.shape[0] *0.75)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1:7]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1:7] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  from sklearn.model_selection import TimeSeriesSplit
  # Cross-validated for time series
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  param_grid = {
      'hidden_layer_sizes': [(4,6,1),(2,6,1),(6,12,1),(6,18,1)], # MLP layers
      'max_iter': [ 100, 150, 200], # maximum iterations
      'activation': [ 'relu','identity'], # activation function
      'solver': ['adam'], # weight optimization algorithm
      'alpha': [0.0001, 0.001, 0.05],  # alpha strength of regularization
  }
  
  from sklearn.neural_network import MLPRegressor
  modelo = MLPRegressor(random_state=0)
  
  grid = GridSearchCV(modelo, param_grid, n_jobs= -1,scoring='neg_mean_absolute_percentage_error', cv=ts_cv, verbose=1)
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))  

  # Stores the training execution end time
  Hora_Fim = time.time()  

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio  

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)  

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))  

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)  

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100   

  salvar_resultado(sk_ponto, resultado, MSE, RMSE, MAE, MAPE, Duracao)

In [66]:
#selects the identification points of each reservoir
df_unique_pontos = dataset['SK_PONTO'].unique()

#create file to results
criar_arquivo_resultado()

for sk in np.array(df_unique_pontos):
    print('forecast for sk_ponto = ',sk)
    df_ponto = dataset[dataset['SK_PONTO']==sk]
    df_ponto = df_ponto.drop('SK_PONTO',axis=1)
    grid = previsao_MLP(sk, df_ponto)

forecast for sk_ponto =  1
Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[1, "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (6, 12, 1), 'max_iter': 200, 'solver': 'adam'}", 4897656.338495648, 2213.0649196297086, 679.030729274391, 18.724169271767874, 22.522935152053833]
forecast for sk_ponto =  4
Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[4, "{'activation': 'identity', 'alpha': 0.05, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 200, 'solver': 'adam'}", 2957203.840207362, 1719.652243974741, 359.5431522957151, 9.167545467052877, 25.230820655822754]
forecast for sk_ponto =  5
Fitting 5 folds for each of 72 candidates, totalling 360 fits


...

['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[9, "{'activation': 'identity', 'alpha': 0.05, 'hidden_layer_sizes': (6, 18, 1), 'max_iter': 150, 'solver': 'adam'}", 855745.9607123354, 925.0653818581341, 136.4443080124638, 4.698277077441014, 20.555655241012573]
forecast for sk_ponto =  11
Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[11, "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (6, 12, 1), 'max_iter': 100, 'solver': 'adam'}", 1808905.7476666127, 1344.9556675469316, 404.2145775078534, 4.33884415860266, 34.684731006622314]
forecast for sk_ponto =  12
Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[12, "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 150, 'solver': 'adam'}", 664203.6736511225, 814.9869162453606, 236.02164618970846, 6.4838160267414136, 35.395771503448486]
forecast for sk_ponto =  13
Fitting 5 folds for each of 72 candidates, totalling 360 fits
['SK_PONTO', 'Best Params', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[13, "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 150, 'solver': 'adam'}", 111480720.3006183, 10558.443081279469, 2843.4736437686006, 25.921984545506856, 34.010984659194946]
